"""
Created on Sun Oct 23 14:18:09 2016

@author: Vaisakh <vaisakhs.shaj@gmail.com>
"""

References
++++++++++

https://www.udacity.com/course/deep-learning--ud730
http://info.usherbrooke.ca/hlarochelle/neural_networks/content.html
http://stackoverflow.com/questions/34727431/tensorflow-on-jupyter-cant-restore-variables

Importing Necessary Librareis

In [ ]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle


class FLAGS:
    def __init__(self):
        self.randomInit = True
        self.b = 0

Hyperparameters and other Inputs necessary for training the model

In [ ]:
FLAG =FLAGS()
FLAG.randomInit = True

pickle_file='/home/dnnlab/vaisakh/anaconda3/Code/Udacity/notMNIST.pickle'
checkPointPath='/home/dnnlab/vaisakh/DNN/ImageModels/NOTMNIST/DNN2layer/961-560.ckpt'
checkPointFolder='/home/dnnlab/vaisakh/DNN/ImageModels/NOTMNIST/DNN2layer'
image_size = 28
num_labels = 10

batch_size = 1000
H=[ 961, 650] #hidden layer size
#print(num_labels)
lam=0.0005
num_steps = 20001

Load data and labels

In [ ]:
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)



In [ ]:
Preprocessing

In [ ]:
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


In [ ]:
Define Tensorflow Graph

In [ ]:
def predictNeural(X,w1,w2,w3,b1,b2,b3):
     layer2= tf.matmul(X, w1) + b1
     layer2out = tf.nn.relu(layer2)
     layer3 = tf.matmul(layer2out, w2) + b2
     layer3out = tf.nn.relu(layer3)
     layer4 = tf.matmul(layer3out, w3) + b3
     layer4out = tf.nn.softmax(layer4)
     return layer4out

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [ ]:
graph = tf.Graph()
with graph.as_default():
  #with tf.device('/gpu:1'):
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                  shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    
# Variables.
    weights1 = tf.Variable(
    tf.truncated_normal([input_size, H[0]]),name="weights1")
    biases1 = tf.Variable(tf.zeros([H[0]]),name="biases1")
    weights2 = tf.Variable(
        tf.truncated_normal([H[0] , H[1]]),name="weights2")
    biases2 = tf.Variable(tf.zeros([H[1]]),name="biases2")
    weights3 = tf.Variable(
        tf.truncated_normal([H[1] , num_labels]),name="weights3")
    biases3 = tf.Variable(tf.zeros([num_labels]),name="biases3")
  
  
  # Training computation.
    X=tf_train_dataset
    layer2 = tf.matmul(X, weights1) + biases1
    layer2out = tf.nn.relu(layer2)
#layer2out = tf.nn.dropout(layer2out,0.9)
    layer3 = tf.matmul(layer2out, weights2) + biases2
    layer3out = tf.nn.relu(layer3)
#layer3out = tf.nn.dropout(layer3out,0.9)
    layer4 = tf.matmul(layer3out, weights3) + biases3
    output = tf.nn.relu(layer4)

    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(layer4, tf_train_labels)) + lam*tf.nn.l2_loss(weights1) + lam*tf.nn.l2_loss(weights2) + lam*tf.nn.l2_loss(weights3)

# Optimizer.
    optimizer = tf.train.AdagradOptimizer(0.5).minimize(loss)

# Predictions for the training, validation, and test data.
    train_prediction = output
    X=tf_valid_dataset
    valid_prediction = predictNeural(X,weights1,weights2,weights3,biases1,biases2,biases3)
    X=tf_test_dataset
    test_prediction = predictNeural(X,weights1,weights2,weights3,biases1,biases2,biases3)


Executing Tensorflow Graph

In [ ]:
with tf.Session(graph=graph) as session:
    saver=tf.train.Saver(var_list={"biases1": biases1, "weights1": weights1, "biases2": biases2, 
                                   "weights2": weights2 ,"biases3": biases3, 
                                   "weights3": weights3 })
    tf.initialize_all_variables().run()
    if FLAG.randomInit:
        print("Initialized")
    else:
        saver.restore(session,tf.train.latest_checkpoint
                (checkPointFolder))
    print(session.run(weights1))
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  
        if(step==num_steps-1):
            save_path=saver.save(session,checkPointPath)
    print(session.run(weights1))   
    